In [ ]:
%pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.9.1+cpu
Uninstalling torch-2.9.1+cpu:
  Successfully uninstalled torch-2.9.1+cpu
Found existing installation: torchvision 0.24.1+cpu
Uninstalling torchvision-0.24.1+cpu:
  Successfully uninstalled torchvision-0.24.1+cpu
Found existing installation: torchaudio 2.9.1+cpu
Uninstalling torchaudio-2.9.1+cpu:
  Successfully uninstalled torchaudio-2.9.1+cpu
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip cache purge

Files removed: 759 (418.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 47.1 MB/s  0:00:140m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 54.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 53.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.7 MB/s  0:00:003.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.3 MB/s  0:00:00 63.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 52.9 MB/s  0:00:120m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 59.8 MB/s  0:00:060m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 61.5 MB/s  0:00:0164.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version used by PyTorch: {torch.version.cuda}")
else:
    print("GPU not detected — check drivers or restart the kernel.")

PyTorch version: 2.5.1+cu121
CUDA available: True
GPU name: NVIDIA RTX A6000
CUDA version used by PyTorch: 12.1


In [ ]:
# -*- coding: utf-8 -*-
"""
Urdu Toxic Span Detection - GPU version (RTX A6000)
"""

# ===============================
# 0. Imports
# ===============================
import os
from ast import literal_eval
import numpy as np
import pandas as pd

# Remove any GPU-hiding environment variables
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # REMOVED
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Hide TensorFlow warnings (if any)

import torch
from datasets import Dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)

# ===============================
# Device setup - Use GPU if available
# ===============================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Using device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

# ===============================
# 1. Config & Paths
# ===============================
DATA_CSV = r"/home/FajarInaya/Downloads/UrduToxicSpanDataset_Attempt1.csv"
SAVE_DIR = r"/home/FajarInaya/Downloads/saved folders atttempt 1"
MODEL_NAME = "xlm-roberta-base"
MAX_LENGTH = 128
TRAIN_BATCH_SIZE = 8  # You can increase to 16 or 32 on A6000 if VRAM allows
EVAL_BATCH_SIZE = 8
NUM_EPOCHS = 5
LEARNING_RATE = 3e-5

# ===============================
# 2. Load dataset
# ===============================
df = pd.read_csv(DATA_CSV)

def ensure_list(example):
    if isinstance(example["tokens"], str):
        example["tokens"] = literal_eval(example["tokens"])
    if isinstance(example["BIO_tags"], str):
        example["BIO_tags"] = literal_eval(example["BIO_tags"])
    return example

dataset = Dataset.from_pandas(df)
dataset = dataset.map(ensure_list)

# ===============================
# 3. Label mappings
# ===============================
label_list = ["O", "B-Toxic", "I-Toxic"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

# ===============================
# 4. Tokenizer
# ===============================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ===============================
# 5. Tokenize and align labels
# ===============================
def tokenize_and_align_labels(example):
    tokens = example["tokens"]
    labels = example["BIO_tags"]
    tokenized_input = tokenizer(
        tokens,
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )
    word_ids = tokenized_input.word_ids()
    label_ids = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx == previous_word_idx:
            label_ids.append(-100)  # Subword gets -100
        else:
            label_ids.append(label2id[labels[word_idx]])
        previous_word_idx = word_idx
    tokenized_input["labels"] = label_ids
    return tokenized_input

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False)
tokenized_dataset = tokenized_dataset.remove_columns(["tokens", "BIO_tags"])

# ===============================
# 6. Split dataset
# ===============================
dataset_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ===============================
# 7. Load model
# ===============================
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)
model.to(DEVICE)  # Moves model to GPU

# ===============================
# 8. Training arguments (GPU enabled)
# ===============================
training_args = TrainingArguments(
    output_dir="./toxic_model_results",
    do_eval=True,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_dir='./logs',
    save_total_limit=2,
    report_to="none",
    # no_cuda=True  # REMOVED - this was forcing CPU!
)

# ===============================
# 9. Data collator & metrics
# ===============================
data_collator = DataCollatorForTokenClassification(tokenizer)

seqeval = evaluate.load("seqeval")

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=2)
    true_labels = [
        [id2label[l] for l in label if l != -100]
        for label in labels
    ]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# ===============================
# 10. Trainer
# ===============================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# ===============================
# 11. Train model
# ===============================
trainer.train()

# ===============================
# 12. Save model & tokenizer
# ===============================
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print(f"Training complete. Model saved at {SAVE_DIR}")

/home/FajarInaya/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
Using device: cuda
GPU name: NVIDIA RTX A6000


Map: 100%|███████████████████████| 14342/14342 [00:05<00:00, 2746.78 examples/s]
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_200700/1951711029.py:174: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.136700
1000,0.099200
1500,0.096500
2000,0.077000
2500,0.078100
3000,0.072500
3500,0.063300
4000,0.063800
4500,0.049200
5000,0.046900


Training complete. Model saved at /home/FajarInaya/Downloads/saved folders atttempt 1


In [ ]:
# ===============================
# ✅ Step 7: Urdu Toxic Span Detection
# Evaluation + Inference
# (GPU-ENABLED – aligned with training on RTX A6000)
# ===============================
import torch
import numpy as np
import json
from collections import Counter
from transformers import AutoTokenizer, AutoModelForTokenClassification

# -------------------------------
# 1. Device (Use GPU – consistent with training)
# -------------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Using device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

# -------------------------------
# 2. Load fine-tuned model
# -------------------------------
model_path = SAVE_DIR  # SAME path used during training
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)
model.to(DEVICE)
model.eval()
print("✅ Model loaded successfully on GPU")

# -------------------------------
# 3. Evaluation on test set (Trainer-based)
# -------------------------------
# Note: trainer is from the training script and already has the model on GPU
print("\n📊 Evaluating model on test set...")
eval_results = trainer.evaluate()
print("\n🎯 Evaluation Results:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f" {key}: {value:.4f}")
    else:
        print(f" {key}: {value}")

# -------------------------------
# 4. Inference function (WORD-LEVEL FIXED + GPU)
# -------------------------------
def predict_toxic_spans(text):
    """
    Predict toxic spans in Urdu text using BIO tagging.
    Returns list of toxic spans.
    """
    words = text.strip().split()
    encoding = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        max_length=128,
        padding="max_length"
    )
    encoding = {k: v.to(DEVICE) for k, v in encoding.items()}

    with torch.no_grad():
        outputs = model(**encoding)
        predictions = torch.argmax(outputs.logits, dim=2)[0].cpu().numpy()

    word_ids = encoding.word_ids(batch_index=0)  # Updated for newer Transformers versions
    predicted_labels = [model.config.id2label[p] for p in predictions]

    toxic_spans = []
    current_span = None
    previous_word_id = None

    for label, word_id in zip(predicted_labels, word_ids):
        if word_id is None:
            continue
        if word_id == previous_word_id:
            continue  # Skip subwords (already handled by word_ids repeat)

        word = words[word_id]

        if label.startswith("B-"):
            if current_span:
                toxic_spans.append(current_span)
            current_span = {
                "label": label[2:],
                "words": [word]
            }
        elif label.startswith("I-") and current_span and current_span["label"] == label[2:]:
            current_span["words"].append(word)
        else:
            if current_span:
                toxic_spans.append(current_span)
                current_span = None

        previous_word_id = word_id

    if current_span:
        toxic_spans.append(current_span)

    return [
        {
            "text": " ".join(span["words"]),
            "label": span["label"]
        }
        for span in toxic_spans
    ]

# -------------------------------
# 5. Test on Urdu example sentences
# -------------------------------
test_sentences = [
    "تم بہت احمق ہو اور کوئی تمہیں پسند نہیں کرتا",
    "یہ ایک عام جملہ ہے جس میں کوئی زہریلا پن نہیں ہے",
    "چپ کرو تم بیوقوف ہو",
    "میں آپ کی رائے سے احترام کے ساتھ اختلاف کرتا ہوں",
    "اپنے آپ کو مار ڈالو ہارے ہوئے",
    "تم واقعی میں کمینے ہو",
    "آپ کا دن اچھا گزرے",
    "کنجر کی اولاد تمہاری ماں کو بھی شرم نہیں آتی",
]

print("\n🧪 Urdu Sample Testing:\n")
for i, sentence in enumerate(test_sentences, 1):
    print(f"مثال {i}: {sentence}")
    spans = predict_toxic_spans(sentence)
    if spans:
        print(" ⚠️ زہریلے الفاظ:")
        for span in spans:
            print(f" - '{span['text']}' [{span['label']}]")
    else:
        print(" ✅ کوئی زہریلا پن نہیں ملا")
    print()

# -------------------------------
# 6. Save predictions on test set
# -------------------------------
print("\n💾 Saving test-set predictions...")
test_predictions = []

# Note: eval_dataset.indices gives original dataframe indices
for idx in range(len(eval_dataset)):
    original_idx = eval_dataset.indices[idx]
    original_text = df.iloc[original_idx]["text"]  # Assuming your original df has a "text" column
    spans = predict_toxic_spans(original_text)
    test_predictions.append({
        "id": original_idx,  # Using original index for traceability
        "text": original_text,
        "toxic_spans": spans,
        "num_toxic_spans": len(spans)
    })

with open("urdu_test_predictions.json", "w", encoding="utf-8") as f:
    json.dump(test_predictions, f, indent=2, ensure_ascii=False)

print("✅ Predictions saved to urdu_test_predictions.json")

# -------------------------------
# 7. Summary statistics
# -------------------------------
total_texts = len(test_predictions)
texts_with_toxicity = sum(p["num_toxic_spans"] > 0 for p in test_predictions)
total_toxic_spans = sum(p["num_toxic_spans"] for p in test_predictions)

print("\n📈 Summary Statistics:")
print(f" Total texts analyzed: {total_texts}")
print(f" Texts with toxicity: {texts_with_toxicity} ({texts_with_toxicity/total_texts*100:.1f}%)")
print(f" Total toxic spans found: {total_toxic_spans}")
print(f" Avg toxic spans per text: {total_toxic_spans/total_texts:.2f}")

# -------------------------------
# 8. Label distribution
# -------------------------------
all_labels = [span["label"] for pred in test_predictions for span in pred["toxic_spans"]]
label_counts = Counter(all_labels)

print("\n🏷️ Label Distribution:")
if label_counts:
    for label, count in label_counts.most_common():
        print(f" {label}: {count}")
else:
    print(" No toxic spans detected in test set.")

print("\n✅ Evaluation & Inference Complete!")

CUDA available: True
Using device: cuda
GPU name: NVIDIA RTX A6000


The tokenizer you are loading from '/home/FajarInaya/Downloads/saved folders atttempt 1' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ Model loaded successfully on GPU

📊 Evaluating model on test set...



🎯 Evaluation Results:
 eval_loss: 0.1183
 eval_precision: 0.5961
 eval_recall: 0.6074
 eval_f1: 0.6017
 eval_accuracy: 0.9741
 eval_runtime: 5.7833
 eval_samples_per_second: 496.0830
 eval_steps_per_second: 62.0750
 epoch: 5.0000

🧪 Urdu Sample Testing:

مثال 1: تم بہت احمق ہو اور کوئی تمہیں پسند نہیں کرتا


AttributeError: 'dict' object has no attribute 'word_ids'